In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv
import os
import json
import requests



In [9]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [3]:
%pwd

'd:\\Study_material\\Production Copilot\\LLMCopilot'

In [2]:
import os
os.chdir("../")

In [6]:
extracted_data=load_pdf_file(data='Data/')

In [10]:
# Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks




In [11]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 1028


In [12]:
# Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [13]:
embeddings = download_hugging_face_embeddings()

C:\Users\chaud\AppData\Local\Temp\ipykernel_25108\2279459995.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
C:\Users\chaud\AppData\Roaming\Python\Python310\site-packages\pydot\core.py:17: UserWarning: `pydot` could not import `dot_parser`, so `pydot` will be unable to parse DOT files. The error was:  No module named 'pyparsing'
  warnings.warn(
c:\Users\chaud\.conda\envs\llmcopilot\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be d

In [14]:
query_result = embeddings.embed_query("Error motor")
print("Length", len(query_result))

Length 384


In [15]:
# Environment Variables
load_dotenv()
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OLLAMA_API_KEY = os.environ.get('OLLAMA_API_KEY')


In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "llmcopilotlama"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [17]:
# Embed the chunks into Pinecone
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

# Load existing Pinecone Index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})



In [18]:
retrieved_docs = retriever.invoke("What is error motor X?")

In [19]:
retrieved_docs

[Document(id='02b738f8-2724-4fa5-88b6-a550f0ab467a', metadata={'page': 0.0, 'source': 'Data\\Descriptions of errors.pdf'}, page_content='Error Motor X  \nA failure in the width control system would mean that the machine is unable to correctly \nadjust for the size of the PCB, which could lead to improper alignment during inspection, \nand potentially, errors in the X or Y motor as the camera cannot correctly mo ve over the \nPCB if it is not properly positioned.  \nThis error typically relates to an issue with the motor responsible for moving the inspection'),
 Document(id='df37c6b1-c7e3-4d1d-b133-849d6beee4a6', metadata={'page': 0.0, 'source': 'Data\\Descriptions of errors.pdf'}, page_content='Error Motor X  \nA failure in the width control system would mean that the machine is unable to correctly \nadjust for the size of the PCB, which could lead to improper alignment during inspection, \nand potentially, errors in the X or Y motor as the camera cannot correctly mo ve over the \nPCB 

In [27]:
# Using Ollama model to answer questions
class OllamaClient:
    def __init__(self, base_url, api_key):
        self.base_url = base_url
        self.headers = {
            'Authorization': f'Bearer {api_key}',
            'Content-Type': 'application/json',
        }

    def chat(self, model, prompt, temperature=0.4, max_tokens=500):
        payload = {
            "model": model,
            "prompt": prompt,
            "temperature": temperature,
            "max_tokens": max_tokens
        }
        response = requests.post(f'{self.base_url}/v1/chat', headers=self.headers, data=json.dumps(payload))
        return response

# Initialize Ollama Client
ollama_client = OllamaClient(base_url="http://workstation.ferienakademie.de:11434", api_key=OLLAMA_API_KEY)

In [28]:


# System Prompt for Troubleshooting Production Line Machines
system_prompt = (
    "You are an AI assistant specialized in troubleshooting production line machines. Your task is to help the user resolve any issues they're experiencing with the production line. "
    "Based on the user's input, logs, and context, please: 1. Identify the problem and its potential causes. 2. Provide a clear, step-by-step solution for the user to follow. 3. Explain your reasoning, referencing specific parts of the logs or context that informed your diagnosis. 4. If applicable, mention any protocols or documentation sources you used. 5. Ask the user to confirm if the solution works, and offer to explore alternative solutions if needed."
    "Remember to be clear, concise, short, and user-friendly in your explanations. If you need more information to diagnose the problem accurately, ask the user specific questions. Don't assume a problem if the user didn't mention it. In this case, explain how you can help the user."
    "When answering, use context provided by the tools available to you. When making statements, always reference the source, i.e., the data returned by the tools. Do not make statements unless you have a tool response to back them up. You may call multiple tools at once."
    "answer concise."
    "\n\n"
    "{context}"
)


In [29]:
# Generate prompt for question
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [30]:
# Example Query
question = "Tell me about Error motor X and how can I solve it?"



In [32]:
# If you have no actual context, just provide an empty string
context_data = ""  # Set this to actual context data if available, or keep it as empty

# Generate prompt by filling in the context
prompt_filled = prompt.format(input=question, context=context_data)

# Send request to Ollama model
response = ollama_client.chat(model="llama3.1", prompt=prompt_filled)

In [33]:
response

<Response [404]>